In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


## Format Data

## Make Keras Model

In [ ]:
def create_model(dropout_rate = 0.15, n=100, activations = 'relu', optimizers='adam'):
    
    model = Sequential()
    
    model.add(Dense(n, input_dim=68))
    model.add(Activation(activations))
    model.add(Dropout(dropout_rate))
    model.add(Dense(100))
    model.add(Activation(activations))
    model.add(Dense(10, activation='softmax'))    
    model.compile(loss='mean_squared_error', optimizer=optimizers, metrics=['accuracy'])

    # Printing a summary of the layers and weights in your model
    model.summary()
    
    return model


## Fit Model

In [ ]:
seed = 0
np.random.seed(seed)
neural_network_clf = KerasClassifier(build_fn=create_model, verbose=0, batch_size=32)

neural_network_parameters = {'dropout_rate':[0.15], 'n':[45],
                             'activations':['relu'], 'epochs':[10]}

neural_network_gsc = GridSearchCV(estimator=neural_network_clf, param_grid=neural_network_parameters, cv=3, 
                                return_train_score=True)

neural_network_gsc.fit(X_train, Y_train, verbose=1)


## Results

In [ ]:
print('best estimator: ' + str(neural_network_gsc.best_estimator_))
print('best score: ' + str(neural_network_gsc.best_score_))
print('best parameters: ' + str(neural_network_gsc.best_params_))

print('In sample error: ' + str(neural_network_gsc.best_estimator_.score(X_train, Y_train)))
neural_network_predictions = neural_network_gsc.best_estimator_.predict(X_train)